In [2]:
from binascii import hexlify, unhexlify
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script
from script import Script
from tx import TxIn, TxOut, Tx

tx_ins = []
prev_tx = unhexlify('0025bc3c0fa8b7eb55b9437fdbd016870d18e0df0ace7bc9864efc38414147c8')
tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=0,
            script_sig=b'',
            sequence=0xffffffff,
        ))
tx_outs = []
h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
tx_outs.append(TxOut(
    amount=int(0.99*100000000),
    script_pubkey=p2pkh_script(h160),
))
h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
tx_outs.append(TxOut(
    amount=int(0.1*100000000),
    script_pubkey=p2pkh_script(h160),
))
tx = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)
sighash = 1
z = tx.hash_to_sign(0, sighash)
pk = PrivateKey(secret=8675309)
der = pk.sign(z).der()
sig = der + bytes([sighash])
sec = unhexlify(pk.point.sec())
script_sig = bytes([len(sig)]) + sig + bytes([len(sec)]) + sec
script_sig = bytes([len(script_sig)]) + script_sig
tx.tx_ins[0].script_sig = Script.parse(script_sig)
print(hexlify(tx.serialize()))

TypeError: hash_to_sign() takes 2 positional arguments but 3 were given

In [1]:
from helper import encode_base58_checksum
from binascii import unhexlify
print(encode_base58_checksum(b'\x6f'+unhexlify('74d691da1574e6b3c192ecfb52cc8984ee7b6c56')))



mrAjisaT4LXL5MzE81sfcDYKU3wqWSvf9q


In [2]:
from binascii import hexlify, unhexlify
from helper import double_sha256

z = int(hexlify(double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))), 16)

print(hex(z))

0xe71bfa115715d6fd33796948126f40a8cdd39f187e4afb03896795189fe1423c


In [3]:
from binascii import hexlify, unhexlify
from ecc import S256Point, Signature
from helper import double_sha256

z = int(hexlify(double_sha256(unhexlify('0100000001868278ed6ddfb6c1ed3ad5f8181eb0c7a385aa0836f01d5e4789e6bd304d87221a000000475221022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb702103b287eaf122eea69030a0e9feed096bed8045c8b98bec453e1ffac7fbdbd4bb7152aeffffffff04d3b11400000000001976a914904a49878c0adfc3aa05de7afad2cc15f483a56a88ac7f400900000000001976a914418327e3f3dda4cf5b9089325a4b95abdfa0334088ac722c0c00000000001976a914ba35042cfe9fc66fd35ac2224eebdafd1028ad2788acdc4ace020000000017a91474d691da1574e6b3c192ecfb52cc8984ee7b6c56870000000001000000'))), 16)

point = S256Point.from_sec(unhexlify('022626e955ea6ea6d98850c994f9107b036b1334f18ca8830bfff1295d21cfdb70'))
r = 0x00dc92655fe37036f47756db8102e0d7d5e28b3beb83a8fef4f5dc0559bddfb94e
s = 0x5a36d4e4e6c7fcd16658c50783e00c341609977aed3ad00937bf4ee942a89937
sig = Signature(r,s)
print(point.verify(z, sig))


True


In [4]:
from script import ScriptTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(ScriptTest())
unittest.TextTestRunner().run(suite)


..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [1]:
from tx import TxTest
import unittest

suite = unittest.TestLoader().loadTestsFromModule(TxTest())
unittest.TextTestRunner().run(suite)


.E......EEEEEE
ERROR: test_hash_to_sign (tx.TxTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jimmy/pb-exercises/session5/tx.py", line 349, in test_hash_to_sign
    self.assertEqual(tx.hash_to_sign(0), want)
  File "/home/jimmy/pb-exercises/session5/tx.py", line 89, in hash_to_sign
    result = alt_tx.serialize() + int_to_little_endian(1, 4)
  File "/home/jimmy/pb-exercises/session5/tx.py", line 58, in serialize
    result += tx_in.serialize()
  File "/home/jimmy/pb-exercises/session5/tx.py", line 215, in serialize
    binary = self.script_sig.serialize()
AttributeError: 'bytes' object has no attribute 'serialize'

ERROR: test_sec (tx.TxTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/jimmy/pb-exercises/session5/tx.py", line 302, in test_sec
    self.assertEqual(hexlify(tx.tx_ins[0].sec()), want)
AttributeError: 'TxIn' object has no 

<unittest.runner.TextTestResult run=14 errors=7 failures=0>